


# The Art of Analyzing Big Data - Final Project
###Israel Antennas Dataset Project 

In this project, we explored the distribution of cellular antennas across Israel and specifically their environmental effects in terms of radiation and landscape "pollution". We analyzed the data along with socio-economic data to see if there is a link between socio-economic levels and the environmental effects of the antennas. To accomplish this analysis, we used three types of datasets: 1. Antennas data that describe the location of the antenna and its radiation effects, the data was taken from the Israel Ministry of environmental protection. 2. Socio-economic data received from the Israel Central Bureau of Statistics describe the socio-economic details and ranking of each statistical areas which are cities and towns areas or sub-areas, we mainly used the cluster ranking of the statistical regions. 3. Geographic data of each statistical area published by the Israeli Mapping Center was used to determine the distribution of antennas across statistical areas. We matched each antennas location to its statistical area so we can further analyze the link between antennas and socio-economic levels.

## Connecting to data sources in google colab, installing libraries and loading data

In [ ]:
#read file from sharefolder
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
# installing libraries
!pip install turicreate
!pip install geopandas
!pip install folium
!pip install plotly_express
!pip install geopy
!pip install itm


In [ ]:
# importing libraries
import numpy as np
import turicreate as tc
import pandas as pd
import geopandas
import folium
from folium.plugins import HeatMapWithTime,HeatMap
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
from shapely.geometry import shape, Point
import itm
from tqdm import tqdm_notebook as tqdm
import re
import dateutil
from dateutil import relativedelta
import plotly_express as px

In [ ]:
# loading socio economic data published by the Israel Central Bureau of Statistics 
socio_economic_df = pd.read_excel("/content/drive/My Drive/bigdata_project_data/socio_economic_index_2008.xls")
# generating a unique key for each statistical area by concatinating the statistical area and the city/town/settlement code
socio_economic_df['area'] = socio_economic_df['semel'].astype(str) + socio_economic_df['statistical_area'].astype(str)
socio_economic_df.dropna(inplace=True)

In [ ]:
socio_economic_df.head()

In [ ]:
# loading antennas data published by Israel ministry of enviormental protection
Antennas = pd.read_excel("/content/drive/My Drive/bigdata_project_data/radiation_non_ionizing_shidur_selulariim_peilim_1.xlsx")

In [ ]:
Antennas.head()

In [ ]:
# loading the polygons of statistical areas published by the Israeli Mapping Center
area_polygons = geopandas.read_file("/content/drive/My Drive/bigdata_project_data/data/stat_2008_NEW_04Nov_1335_fix.shp" )
area_polygons['area'] = area_polygons["SEMEL_YISH"].astype(str)+area_polygons["STAT08"].astype(str) 
area_polygons = area_polygons.loc[area_polygons['area'].isin(socio_economic_df['area'])]
area_polygons.dropna(inplace=True)
plygon_json = area_polygons.to_json()

In [ ]:
area_polygons.head()

In [ ]:
# iterating over each antenna location and checking in which statistical area polygon does it fit
def get_area_from_itm(y_itm,x_itm,area_polygons):
    coord_y,coord_x = itm.itm2wgs84(y_itm, x_itm )
    # print(coord_y,coord_x)
    point = Point(coord_x, coord_y)
    for area,polygon in zip(area_polygons['area'].tolist(),area_polygons['geometry'].tolist()):
        polygon = shape(polygon)
        if polygon.contains(point):
          return area

# areas = []
# for antenna_y,antenna_x in tqdm(zip(Antennas['Y_ITM'],Antennas['X_ITM'])):
#     areas.append(get_area_from_itm(antenna_y,antenna_x,area_polygons))

# Antennas['area'] = areas
# Antennas.to_csv('antennas_with_areas')

In [ ]:
# we used for the analysis only statistical areas that have antennas
antennas_with_areas = pd.read_csv('/content/drive/My Drive/bigdata_project_data/antennas.csv')
antennas_with_areas.dropna(inplace=True)
antennas_with_areas['area'] = antennas_with_areas['area'].astype(int).astype(str)
antennas_with_areas['תוצאה מירבית ב% ביחס לסף הבריאות'] = antennas_with_areas['תוצאה מירבית ב% ביחס לסף הבריאות'].apply(lambda x: re.sub('\.+','.',x)).astype(float)


In [ ]:
def get_datetime(dt_str):
    try:
        return dateutil.parser.parse(dt_str)
    except:
        return None


antennas_with_areas['תאריך היתר הקמה'] = antennas_with_areas['תאריך היתר הקמה'].apply(lambda dt_str: get_datetime(dt_str))
antennas_with_areas['Month_permit'] = antennas_with_areas['תאריך היתר הקמה'].apply(lambda dt: dt.month)
antennas_with_areas['Year_permit'] = antennas_with_areas['תאריך היתר הקמה'].apply(lambda dt: dt.year)

antennas_with_areas['תאריך היתר הפעלה'] = antennas_with_areas['תאריך היתר הפעלה'].apply(lambda dt_str: get_datetime(dt_str))
antennas_with_areas['Month_activate'] = antennas_with_areas['תאריך היתר הפעלה'].apply(lambda dt: dt.month)
antennas_with_areas['Year_activate'] = antennas_with_areas['תאריך היתר הפעלה'].apply(lambda dt: dt.year)
antennas_with_areas['gap'] = antennas_with_areas['Year_activate']  - antennas_with_areas['Year_permit']

antennas_with_areas['gap_DT'] = [relativedelta.relativedelta(hfala ,hakama).years*12+relativedelta.relativedelta(hfala ,hakama).months for hfala,hakama in zip(antennas_with_areas['תאריך היתר הפעלה'] , antennas_with_areas['תאריך היתר הקמה'])]
antennas_with_cluster = pd.merge(antennas_with_areas,socio_economic_df[['area','cluster']],on='area')
antennas_with_cluster.head()

In [ ]:
# merging the antennas dataset with the partly socio economic dataset
antennas_with_cluster = pd.merge(antennas_with_areas,socio_economic_df[['area','cluster','population ']],on='area')

## Project Assumptions

There are few assumptions that we learned about antennas and socio-economic ranking before we started analyzing the data:


1.   Antennas radiation measure is worse for humans when it is higher.
2.   There is a strong regulation by the state on antennas radiation.
3.   There is a secondary radiation hazard from using a cellular phone when your phone is far from an antenna. It is more radiating, so more antennas are usually better than a few antennas in terms of phones cellular radiation.
4.   Areas are divided into 19 socio-economic clusters, where 2 is the lowest socio-economy areas, and 20 is the highest socio-economic areas.
5.   We analyzed only statistical areas that have antennas in their jurisdiction


## Socio Economic Data Exploration

### Map of Socio Economic Areas colored by socio economic cluster defined by the Israel Central Bureau of Statistics. 20 is Rich statistical areas and 2 is poor statistical areas

In [ ]:

m = folium.Map(location=[31.803772299985, 35.10324724174089], zoom_start=9)

m.choropleth(
 geo_data=plygon_json,
 name='choropleth',
 data=socio_economic_df,
 columns=['area' , 'cluster'],
 key_on='feature.properties.area',
 fill_color='YlOrRd',
 fill_opacity=0.7,
 line_opacity=0.2,
)
m

### Socio Economic Exploration

#### Figure 1.1

In [ ]:
fig, ax = plt.subplots(figsize=(22,8))
sns.distplot(list(socio_economic_df['cluster']), hist=False, kde=True, kde_kws={"clip": [1,20]})
ax.set(xlabel="Economic Clusters", ylabel="Count")
ax.set_title("Socio-Economic Clusters Disribution", fontsize=20)
plt.show()

It can be seen that most of the common socio economic areas are from clusters 8 - 14

#### Figure 1.2

In [ ]:
fig, ax = plt.subplots(figsize=(22,8))
sns.distplot(list(socio_economic_df['population ']), hist=False)
ax.set(xlabel="Population", ylabel="Percentage")
ax.set_title("Socio-Economic Statistical Area Population Disribution", fontsize=20)
plt.show()

Most statistical areas have populations between 2500 - 5000 people

#### Figure 1.3

In [ ]:
df = socio_economic_df.groupby(['cluster'])['population '].mean().to_frame('population_avg').reset_index()
fig, ax = plt.subplots(figsize=(22,8))
sns.set(style="whitegrid")
ax = sns.barplot(x='cluster', y="population_avg", data=df)
ax.set_title("Average population by Socio Economic Cluster", fontsize=20)
plt.show()

From the population distribution based on clusters we can learn that on average most cluster have the same populations size, except the 5 lower socio economic clusters that have larger population, that 3 highest socio economy cluster tend to have a slightly smaller population.

## Antennas Data Exploration

### Antennas location map

In [ ]:
from folium.plugins import MarkerCluster
def pop_text(r):
    txt =str(r['עוצמת הקרינה'])
    return txt

m = folium.Map(location=[31.803772299985, 35.10324724174089], zoom_start=9, tiles='CartoDB dark_matter')
mc = MarkerCluster()
for index,r in Antennas.iterrows():
    coord_y,coord_x = itm.itm2wgs84(r['Y_ITM'],  r['X_ITM'] )
    mc.add_child(folium.CircleMarker(location=[coord_y, coord_x],
                        radius=5,color="#007849", popup=pop_text(r), parse_html=True))
m.add_child(mc)
m

### Antennas Radiation Analysis




#### Figure 2.1

In [ ]:
fig, ax = plt.subplots(figsize=(22,8))

sns.distplot(list(antennas_with_areas['עוצמת הקרינה']), hist=True, kde=False)
ax.set(xlabel="Radiation Intensity", ylabel="Count")
ax.set_title("Radiation Intensity Distribution", fontsize=20)
ax.xaxis.limit_range_for_scale(0,100)
plt.show()


Most of the antennas radiation is betweeb 0 and 75 where the larger portion of antennas have close to 0 radiation, this make sense as there is a strong regulation of cellular radiation in Israel.

#### Figure 2.2

In [ ]:
fig, ax = plt.subplots(figsize=(22,8))

sns.distplot(list(antennas_with_areas['תוצאה מירבית ב% ביחס לסף הבריאות']), hist=False, kde=True)
ax.set(xlabel="Precentage of Radiation with Respect to the Health Radiation Limit Rates", ylabel="Precentage")
ax.set_title("Precentage of Radiation with Respect to the Health Radiation Limit Rates", fontsize=20)

plt.show()

Another measure we have on antennas radiations is the precentage of radiation with respect to the health radiation limit rates. This measure correlates with the previous radiation measure and as we can see a large portion of the antennas have less then 100 precents of the radiation limit rates. The meausre is between 0 and ~470.


In [ ]:
def revere(x):
  return x[::-1]

df = Antennas.groupby(['סוג אתר'])['סוג אתר'].size().to_frame('count').reset_index()
df = df.sort_values('count')
df['רתא גוס']=df['סוג אתר'].apply(revere)
df
fig, ax = plt.subplots(figsize=(22,8))
sns.set(style="whitegrid")
ax = sns.barplot(x='רתא גוס', y="count", data=df)
ax.set(xlabel='Antenna Type', ylabel="Antennas Count")
ax.set_title("Types of Antennas Distribution", fontsize=20)
plt.show()

In the above bar chart we see the distribution of different types of antennas, Ground Antennas are the most common type of antennas and are the type of antennas that from enviormental prespective "pollut" the landscape more than other antennas type. 

## Analyzing Antennas Data with respect to socio economic clusters

### Figure 3.1

In [ ]:
a_rad_power_mean = antennas_with_cluster.groupby('cluster')['עוצמת הקרינה'].mean().reset_index()
fig, ax = plt.subplots(figsize=(22,8))

sns.barplot(a_rad_power_mean['cluster'].tolist(),a_rad_power_mean['עוצמת הקרינה'].tolist())
ax.set(xlabel="Socio Economic Cluster", ylabel="Radiation Intensity")
ax.set_title("Average Antennas Radiation Intensity by Socio Economic Cluster", fontsize=20)
plt.show()

We can see that the is no correlation between socio economic clusters and antennas radiation but it can be seen that the lowest socio-economic areas have more radiating antennas then other statistical areas. 2 time more highest cluster statistical areas.

### Figure 3.2

In [ ]:
a_rad_power_median = antennas_with_cluster.groupby('cluster')['עוצמת הקרינה'].median().reset_index()
fig, ax = plt.subplots(figsize=(22,8))

sns.barplot(a_rad_power_median['cluster'].tolist(),a_rad_power_median['עוצמת הקרינה'].tolist())
ax.set(xlabel="Socio Economic Cluster", ylabel="Radiation Intensity")
ax.set_title("Meidan Antennas Radiation Intensity by Socio Economic Cluster", fontsize=20)
plt.show()

When examining the median the differnces between the lowest and highest socio economic areas is more apparent


### Figure 3.3

In [ ]:
a_max_percent_mean = antennas_with_cluster.groupby('cluster')['תוצאה מירבית ב% ביחס לסף הבריאות'].mean().reset_index()
fig, ax = plt.subplots(figsize=(22,8))

sns.barplot(a_max_percent_mean['cluster'].tolist(),a_max_percent_mean['תוצאה מירבית ב% ביחס לסף הבריאות'].tolist())
ax.set(xlabel="Socio Economic Cluster", ylabel="Precentage of Radiation with Respect to the Health Radiation Limit Rates")
ax.set_title("Average Precentage of Radiation with Respect to the Health Radiation Limit Rates by Socio Economic Cluster", fontsize=20)
plt.show()

### Figure 3.4

In [ ]:
a_max_percent_median = antennas_with_cluster.groupby('cluster')['תוצאה מירבית ב% ביחס לסף הבריאות'].median().reset_index()
fig, ax = plt.subplots(figsize=(22,8))

sns.barplot(a_max_percent_median['cluster'].tolist(),a_max_percent_median['תוצאה מירבית ב% ביחס לסף הבריאות'].tolist())
ax.set(xlabel="Socio Economic Cluster", ylabel="Precentage of Radiation with Respect to the Health Radiation Limit Rates")
ax.set_title("Median Precentage of Radiation with Respect to the Health Radiation Limit Rates by Socio Economic Cluster", fontsize=20)
plt.show()

Here we can see more that from health prespective the antennas in the lowest socio-economic areas are less safe then the others socio-economic clusters

### Figure 3.5

In [ ]:
mean_count_of_antennas_by_areas = antennas_with_cluster.groupby(['area','cluster']).size().to_frame('size').reset_index().groupby('cluster')['size'].mean().reset_index()
fig, ax = plt.subplots(figsize=(22,8))

sns.barplot(mean_count_of_antennas_by_areas['cluster'].tolist(),mean_count_of_antennas_by_areas['size'].tolist())
ax.set(xlabel="Socio Economic Cluster", ylabel="Average Antennas Count")
ax.set_title("Average Antennas Count by Socio Economic Cluster", fontsize=20)
plt.show()

From the above figure we can see that the better socio economic area the average of antennas is higher, there more then 1.5 times more antennas on average on highest socio economic areas then the lower socio economic areas. This is a crucial analysis as more antennas usually results in less radiation in users phone, which is usually more critical as most phone users live far from cellular towers direct radiation and phone radiation is the most common cellular radiation people are dealing with

### Avergae Data By Statistical Area and then by Statistical Cluster



#### Figure 3.6

In [ ]:
avg_areas_avg = antennas_with_cluster.groupby('area').mean()[['cluster','population ','תוצאה מירבית ב% ביחס לסף הבריאות','עוצמת הקרינה']].groupby('cluster')['תוצאה מירבית ב% ביחס לסף הבריאות','עוצמת הקרינה'].mean().reset_index()
fig, ax = plt.subplots(figsize=(22,8))

sns.barplot(avg_areas_avg['cluster'].astype(int).tolist(),avg_areas_avg['עוצמת הקרינה'].tolist())
ax.set(xlabel="Socio Economic Cluster", ylabel="Radiation Intensity")
ax.set_title("Average Antennas Radiation Intensity first by Statistical Areas and then by Socio Economic Cluster", fontsize=20)
plt.show()

We tried to see if we learn more details from first calculating the average radiation from each statistical areas and only then calculate the average radiation per clusters. We can see that there is no big difference in this approach

#### Figure 3.7

In [ ]:
# plt.bar(avg_areas_avg['cluster'].astype(int).tolist(),avg_areas_avg['תוצאה מירבית ב% ביחס לסף הבריאות'].tolist(), align='center')
fig, ax = plt.subplots(figsize=(22,8))

sns.barplot(avg_areas_avg['cluster'].astype(int).tolist(),avg_areas_avg['תוצאה מירבית ב% ביחס לסף הבריאות'].tolist())
ax.set(xlabel="Socio Economic Cluster", ylabel="Precentage of Radiation with Respect to the Health Radiation Limit Rates")
ax.set_title("Average Antennas Precentage of Radiation with Respect to the Health Radiation Limit Rates first by Statistical Areas and then by Socio Economic Cluster", fontsize=20)
plt.show()

### Figure 3.8

In [ ]:
top_quarter = antennas_with_cluster[antennas_with_cluster['עוצמת הקרינה'] > antennas_with_cluster['עוצמת הקרינה'].quantile(0.90)]

top_q_size_by_cluster = top_quarter.groupby('cluster').size().to_frame('size').reset_index()
top_q_size_by_cluster['precentage_in_top_quarter'] = top_q_size_by_cluster['size']/antennas_with_cluster.groupby('cluster').size().to_frame('size').reset_index()['size']
fig, ax = plt.subplots(figsize=(22,8))

sns.barplot(top_q_size_by_cluster['cluster'].astype(int).tolist(),top_q_size_by_cluster['precentage_in_top_quarter'].tolist())
ax.set(xlabel="Socio Economic Cluster", ylabel="Radiation Intensity")
ax.set_title("Top 10 precent of the most radiating antennas distributed by socio economic clusters", fontsize=20)
plt.show()

We wanted to know how the 10 percent most radiating antennas distributed across the socio economic clusters, as we can see 20% of the most radiating antennas are in the lowest socio economic areas.

### Figure 3.9

In [ ]:
bottom_10 = antennas_with_cluster[antennas_with_cluster['עוצמת הקרינה'] < antennas_with_cluster['עוצמת הקרינה'].quantile(0.10)]

bottom_10_size_by_cluster = bottom_10.groupby('cluster').size().to_frame('size').reset_index()
bottom_10_size_by_cluster['precentage_in_top_quarter'] = bottom_10_size_by_cluster['size']/antennas_with_cluster.groupby('cluster').size().to_frame('size').reset_index()['size']
fig, ax = plt.subplots(figsize=(22,8))

sns.barplot(bottom_10_size_by_cluster['cluster'].astype(int).tolist(),bottom_10_size_by_cluster['precentage_in_top_quarter'].tolist())
ax.set(xlabel="Socio Economic Cluster", ylabel="Radiation Intensity")
ax.set_title("Top 10 precent of the least radiating antennas distributed by socio economic clusters", fontsize=20)
plt.show()

We wantend to learn how the least radiating antennas are distributed, as we can see the best antennas radiation-wise are in the 2 highest socio economic clusters and the least "good" antennas are in the lowest socio economic areas.

In [ ]:
area_antennas_count = antennas_with_cluster.groupby('area').size().to_frame('size').reset_index()
area_antennas_rad = antennas_with_cluster.groupby('area')[['תוצאה מירבית ב% ביחס לסף הבריאות','עוצמת הקרינה']].mean().reset_index()
mean_count_of_antennas_by_areas = pd.merge(area_antennas_count,socio_economic_df[['area','cluster','population ']],on='area',)
mean_count_of_antennas_by_areas = pd.merge(mean_count_of_antennas_by_areas,area_polygons[['area','Shape_Area']],on='area')
rad_by_areas = pd.merge(mean_count_of_antennas_by_areas,area_antennas_rad,on='area')
rad_by_areas['population_antennas_ratio'] = rad_by_areas['population ']/rad_by_areas['size']

rad_by_clusters = rad_by_areas.groupby('cluster').mean().reset_index()
rad_by_clusters['density'] = rad_by_clusters['Shape_Area']/rad_by_clusters['population ']
rad_by_clusters['number_of_antennas_devided_by_density'] = rad_by_clusters['density']/rad_by_clusters['size']
rad_by_clusters['population_antennas_ratio'] = rad_by_clusters['population ']/rad_by_clusters['size']
rad_by_clusters['area_shape_divided_by_number_of_antennas'] = rad_by_clusters['Shape_Area']/rad_by_clusters['size']


### Figure 3.10

In [ ]:
fig, ax = plt.subplots(figsize=(22,8))

sns.barplot(rad_by_clusters['cluster'].astype(int).tolist(),rad_by_clusters['population_antennas_ratio'].tolist())
ax.set(xlabel="Socio Economic Cluster", ylabel="Average Population per 1 Antenna")
ax.set_title("Average Ratio of Population and Number of Antennas in each Socio-Economic Cluster", fontsize=20)
plt.show()

Here we wanted to calculate the ratio between the areas population and the amount of antennas in the areas. In other words, on average how many people 1 antenna serves. We can see there is a constant decrease in the population / antennas ratio as you go up in the socio economic cluster. This is important measure as the more people antennas serves the radiation is usually higher and there is a clear motivation radition wise to lower the population / antennas ratio

### Figure 3.11

In [ ]:
fig, ax = plt.subplots(figsize=(22,8))

sns.barplot(rad_by_clusters['cluster'].astype(int).tolist(),rad_by_clusters['area_shape_divided_by_number_of_antennas'].tolist())
ax.set(xlabel="Socio Economic Cluster", ylabel="Ratio between Statistical Area size and Number of Antennas")
ax.set_title("Ratio of Area size and Number of Antennas distributed by socio economic clusters", fontsize=20)
plt.show()

Here we measured the number of antennas with respect to the size of areas, we can see that on average lower socio economic cluster have less antennas per area which also can result in higher phone radiation as phone emit more radiation when it is far from the antenna.

### Figure 3.12

In [ ]:
count_area = antennas_with_cluster.groupby(['cluster','area']).size().to_frame('size').reset_index()
count_area = count_area.groupby(['cluster'])['area'].size().to_frame('sum_area_by_cluster').reset_index()
df = antennas_with_cluster.groupby(['Year_permit','cluster']).size().to_frame('size').reset_index()

df_merge = pd.merge(df,count_area,on='cluster')
df_merge['norm_by_number_of_areas'] =df_merge['size']/df_merge['sum_area_by_cluster']

fig = px.scatter(df_merge, x="Year_permit", y="cluster", size="norm_by_number_of_areas",
                  title="Average number of permits per area for socio economic cluster in each Year")
plt.show()
fig.show()

The above figure describe the average new antennas permits on each cluster in each year. We can see that there is slightly more antennas permits in the higher socio economic cluster, this can somehow explain why on average there are more antennas in the higher socio economic clusters then the lower ones.

### Figure 3.13

In [ ]:
fig, ax = plt.subplots(figsize=(22,8))
df = antennas_with_cluster.groupby(['cluster'])['gap_DT'].mean().to_frame('av_gap').reset_index()
sns.barplot(df['cluster'].astype(int).tolist(),df['av_gap'].tolist())
ax.set(xlabel="Socio Economic Cluster", ylabel="Number of Months Between Permits to Activation")
ax.set_title("Number of Months Between Permits to Activation distributed by socio economic clusters", fontsize=20)
plt.show()

This figure describes the time in months between an antenna is premited to operate and the time it started operating. There is no a clear differences between most of the clusters. Some lower clusters have relative quick operating time, We need more data and an expert to understand why there is a difference (less regulation, less objections from the population etc)

### Figure 3.14

In [ ]:
antennas_with_cluster['type'] = antennas_with_cluster['סוג אתר']

count_area = antennas_with_cluster.groupby(['cluster','area']).size().to_frame('size').reset_index()
count_area = count_area.groupby(['cluster'])['area'].size().to_frame('sum_area_by_cluster').reset_index()

df = antennas_with_cluster.groupby(['cluster','type']).size().to_frame('count_type').reset_index()

df_merge = pd.merge(df,count_area,on='cluster')

df_merge['norm_by_number_of_areas'] =df_merge['count_type']/df_merge['sum_area_by_cluster']


fig = px.scatter(df_merge, x="cluster", y="type", size="norm_by_number_of_areas",title="Average number of antennas by type per socio economic cluster")

fig.show()




This is a distribution of the type of antennas by socio economic clusters. There is more small antennas in higher socio economic clusters, but this is predictable as those are used in high buildings, offices and malls. There are more ground antennas in higher socio economic areas which are usually "pollut" the landscape more then other antennas.

## Conclusions



*   There are a clearly more radiation from antennas in the lowest socio economic cluster
*   There are a clearly less radiation from antennas in the highest socio economic cluster
*   There more antennas on average the higher the socio economic area cluster, and the ratio between the population and the number of antennas is higher at the lower socio economic clusters, this can carefully implies cellular phone users on lower socio economic cluster suffer from a more radiation from their phones.

